In [3]:
import pandas as pd
from macdlib import *


STOCK_CODE='600125.SH'
df = pd.read_csv(f'./data/{STOCK_CODE}.csv')

df.rename(columns={df.columns[0]:"date"}, inplace=True)
diff, dem ,bar = macd(df)
df['bar']=bar
df['ma5']=ma(df, 5)
df['ma10']=ma(df,10)
df['ma20']=ma(df,20)

df.to_csv(f"./data/temp/{STOCK_CODE}_macd.csv")


blue_bar_width = 6  # 绿柱子多少个才考虑买入



# i = 0
# print(f"index size= {bar.index.size}")

print("===================================")
_, blue_bar = find_successive_bar_area(df)
buy_dates = []
MIN_BLUE_AREA_WIDTH=6

for s, e in blue_bar:
    #print(f"{s}\t{e}")
    if e-s < MIN_BLUE_AREA_WIDTH:
        continue
    else:#把hi值，close取出来
        print(f"{s}\t{e}")
        df2 = df.loc[s:e, ['bar']]
        row_index = df[df.bar==df2.bar.min()].index[0]
        row = df.loc[row_index:row_index, ['date','close','open']]
        buy_row = df.loc[row_index+2:row_index+2, ['date','close','open']]
        #print(buy_row['date'].values[0])
        buy_dates.append(buy_row['date'].values[0])
        

3	11
29	40
53	64
72	82
96	112
133	143
148	164
179	192
196	203
209	220
227	236
271	290
311	333
339	360
366	388
395	407
444	450
460	469
485	491
500	520
564	581
584	590
615	629


In [ ]:
df

In [4]:
_, blues = find_successive_bar_area(df)
blues

[(0, 0),
 (3, 11),
 (29, 40),
 (53, 64),
 (72, 82),
 (96, 112),
 (133, 143),
 (148, 164),
 (179, 192),
 (196, 203),
 (209, 220),
 (227, 236),
 (260, 260),
 (271, 290),
 (311, 333),
 (339, 360),
 (366, 388),
 (395, 407),
 (444, 450),
 (460, 469),
 (485, 491),
 (500, 520),
 (564, 581),
 (584, 590),
 (604, 605),
 (610, 612),
 (615, 629),
 (633, 636)]

In [ ]:
index = df[df['bar']<0].index.array

from itertools import groupby
from operator import itemgetter
data = index # [ 1, 4,5,6, 10, 15,16,17,18, 22, 25,26,27,28]
for k, g in groupby(enumerate(data), lambda iv: iv[0]-iv[1]):
    x = list(map(itemgetter(1), g))
    print(x)
    
index

In [ ]:
# blue_bar
# blue_bar_2 = []
# for a,b in blue_bar:
#     if b-a <MIN_BLUE_AREA_WIDTH:
#         continue
#     else:
#         blue_bar_2.append((a,b))
        
# blue_bar_2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# fig, ax = plt.subplots(1,1, figsize=(20, 7))
# ax.stem(df[90:108].date, df[90:108].bar, use_line_collection=True)
# ax.xaxis.set_tick_params(rotation=45)

# for s, e in blue_bar_2:
#     #print(f"{s}\t{e}")
#     start_i = s-1
#     if start_i<0:
#         start_i = 0
#     end_i = e+2
    
#     start_dt = df[start_i:start_i+1].date.values[0]
#     end_dt = df[end_i:end_i+1].date.values[0]
#     fig, ax = plt.subplots(1,1, figsize=(20, 7))
#     ax.stem(df[start_i:end_i].date, df[start_i:end_i].bar, use_line_collection=True)
#     ax.xaxis.set_tick_params(rotation=45)
#     ax.set_title(f"{start_i}-{end_i}, {start_dt}-{end_dt}")
    

In [ ]:
df[92:103].date
df[df.date=='2019-01-04 11:00:00']

In [ ]:
## 统计一下这几个时间点进入之后，后面16个周期内最高点超过2%的

#先找处后面16周期最低、最高、当前open, close
# for s, e in blue_bar_2:
#     #print(f"{s}\t{e}")
#     start_i = max(0, s-1)
#     end_i = min(e+2, df.size)
    
#     start_dt = df[start_i:start_i+1].date.values[0]
#     end_dt = df[end_i:end_i+1].date.values[0]
    
tj_df = pd.DataFrame()

for date in buy_dates:
    #print(date)
    start_i = df[df.date==date].index[0]
    end_i = min(start_i+4*4, df.size)
    # 计算区间内的最大，最小值
    low = df[start_i:start_i+4].low.min()
    low_date = df[start_i:start_i+4][df.low==low].date.values[0]
    
    # 从low_date 到end_i
    low_date_i = df[df.date==low_date].index[0]
    
    hi = df[low_date_i:end_i].high.max() #一定是从最小值之后才有机会找最大的
    hi_date = df[low_date_i:end_i][df.high==hi].date.values[0]
    
    
    cur_open = df[df.date==date].open.values[0]
    cur_low = df[df.date==date].low.values[0]
    
    buy_limit_price=cur_open*0.99   #向下挂个1%
    s = pd.DataFrame({"date":[date],"cur_open":cur_open,"cur_low":cur_low, "limit_buy_price":buy_limit_price,  "low":[low], "hi":[hi], "chg_pct":[round((hi-low)/low*100,3)], "low_date":[low_date], "hi_date":[hi_date]})
    tj_df= pd.concat([tj_df, s])
    
from datetime import datetime

tj_df['buy'] = tj_df.chg_pct.apply(lambda x: '✓' if x>=2 else "×")

# tj_df['real_low_date_delta'] = tj_df.low_date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))-tj_df.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
# tj_df['real_low_date_delta'] = tj_df.real_low_date_delta.apply(lambda x: x.seconds/3600)

tj_df['buy_price.9'] = tj_df[['low', 'limit_buy_price']].max(axis=1)
tj_df['e_earn_9'] = (tj_df['hi']-tj_df['buy_price.9'])/tj_df['buy_price.9']*100

tj_df['e_earn']=(tj_df['hi']-tj_df['cur_open'])/tj_df['cur_open']*100
tj_df.reset_index(drop=True)


In [ ]:
st_df = pd.DataFrame({
    "e=1%":[tj_df[tj_df.chg_pct>=1].size/tj_df.size],
    "e=1.5%":[tj_df[tj_df.chg_pct>=1.5].size/tj_df.size],
    "e=2%":[tj_df[tj_df.chg_pct>=2].size/tj_df.size],
    "e=3%":[tj_df[tj_df.chg_pct>=3].size/tj_df.size],
})

st_df